# RF6 Impact Attestation Metrics

Spec [here](https://plaid-cement-e44.notion.site/Impact-Attestations-Data-b8b1c79a624c46ad94305c74def64783).

In [1]:
import datetime
import json
import pandas as pd

from scripts.agora import agora_api
from scripts.farcaster import load_farcaster, refresh_farcaster
from scripts.metrics import calculate_metrics

# Load data

In [2]:
def convert_farcaster(farcaster_id):
    return int(farcaster_id['hex'], 16) if farcaster_id else None

In [3]:
#refresh_farcaster()
f = load_farcaster()

FARCASTER_USERS = pd.DataFrame(f)[['fid', 'username']].drop_duplicates().set_index('fid')['username'].to_dict()
FARCASTER_LINKED_ADDRESSES = pd.DataFrame(f)[['fid', 'address']].dropna().groupby('fid')['address'].apply(set).to_dict()
FARCASTER_ADDRESS_TO_ID = pd.DataFrame(f)[['fid', 'address']].dropna().set_index('address')['fid'].to_dict()

In [4]:
delegate_data = agora_api('delegates', params={'limit': 100, 'offset': 0, 'sort': 'voting_power'})
rf6_projects = agora_api('retrofunding/rounds/6/projects')

DELEGATES = [x['address'].lower() for x in delegate_data]
AGORA_IDS = [x['projectId'] for x in rf6_projects]

Fetched a total of 100 items from delegates.
Fetched 100 items, offset now at 100
Fetched a total of 142 items from retrofunding/rounds/6/projects.


In [5]:
rf6_projects_data = [{
    'id': x['id'],
    'applicationId': x['applicationId'],
    'projectId': x['projectId'],
    'name': x['name'],
    'organization': x['organization']['name'] if x.get('organization') else None,
    'applicationCategory': x['applicationCategory']
    }
    for x in rf6_projects
]
rf6_projects_df = pd.DataFrame(rf6_projects_data)
AGORA_NAMES = rf6_projects_df.set_index('projectId')['name'].to_dict()
AGORA_NAMES.update({'0x09261dbfb4dfa7116ee387d17dbbbdab9494de24cae1561ef7f25c8836b59e71': 'Syntra'})
rf6_projects_df.to_csv('data/_local/rf6_projects.csv')

In [6]:
def load_json(json_path):
    with open(json_path, 'r') as f:
        data = json.load(f)
    return data

project_attestation_data = load_json('data/attestations/472.json')
print("Project attestations:", len(project_attestation_data))

app_attestation_data = load_json('data/attestations/609.json')
print("Application attestations:", len(app_attestation_data))

badgeholder_attestation_data = load_json('data/attestations/599.json')
print("Badgeholder attestations:", len(badgeholder_attestation_data))

gov_council_attestation_data = load_json('data/attestations/141.json')
print("Gov Council attestations:", len(gov_council_attestation_data))

impact_attestation_data = load_json('data/attestations/566.json')
print("Impact attestations:", len(impact_attestation_data))

Project attestations: 4335
Application attestations: 407
Badgeholder attestations: 305
Gov Council attestations: 222
Impact attestations: 864


In [7]:
ATTESTER_ADDR = "0xf6872d315cc2e1aff6abae5dd814fd54755fe97c"

df_project_metadata = pd.DataFrame(project_attestation_data)
df_project_metadata = df_project_metadata[df_project_metadata.attester.str.lower() == ATTESTER_ADDR]
df_project_metadata.sort_values(by="timeCreated", ascending=False, inplace=True)
df_project_metadata.drop_duplicates(subset="projectRefUID", keep="first", inplace=True)

PROJECTS = list(df_project_metadata['projectRefUID'].unique())
print("Projects:", len(PROJECTS))

Projects: 3075


In [8]:
df_applications = pd.DataFrame(app_attestation_data)
df_applications = df_applications[df_applications.attester.str.lower() == ATTESTER_ADDR]
df_applications['round'] = df_applications['round'].apply(lambda x: str(x))
df_applications.sort_values(by="timeCreated", ascending=False, inplace=True)
df_applications.drop_duplicates(subset="metadataSnapshotRefUID", keep="first", inplace=True)

latest_metadata = list(df_project_metadata.id.unique())
df_applications = df_applications[df_applications.metadataSnapshotRefUID.isin(latest_metadata)]
print("Applications:", len(df_applications))

Applications: 138


# Clean / filter attestation data

In [9]:
ATTESTERS = []

for a in badgeholder_attestation_data:
    if a['attester'] != '0xE4553b743E74dA3424Ac51f8C1E586fd43aE226F':
        continue
    if a['voterType'] == 'Guest':
        continue

    addr = a['recipient'].lower()
    fid = convert_farcaster(a['farcasterID'])
    username = FARCASTER_USERS.get(fid, '')
    ATTESTERS.append({
        'address': addr,
        'farcaster_id': fid,
        'farcaster_username': username,
        'governance_role': 'Citizen',
        'source': 'attestation_schema_599'
    })
    linked_addresses = FARCASTER_LINKED_ADDRESSES.get(fid, [])
    for addr in linked_addresses:
        ATTESTERS.append({
            'address': addr.lower(),
            'farcaster_id': fid,
            'farcaster_username': username,
            'governance_role': 'Citizen',
            'source': 'farcaster_linked_address'
        })

for addr in DELEGATES:
    fid = FARCASTER_ADDRESS_TO_ID.get(addr, '')
    username = FARCASTER_USERS.get(fid, '')
    ATTESTERS.append({
        'address': addr.lower(),
        'farcaster_id': fid,
        'farcaster_username': username,
        'governance_role': 'Top 100 Delegate',
        'source': 'agora_api'
    })
    linked_addresses = FARCASTER_LINKED_ADDRESSES.get(fid, [])
    for addr in linked_addresses:
        ATTESTERS.append({
            'address': addr.lower(),
            'farcaster_id': fid,
            'farcaster_username': username,
            'governance_role': 'Top 100 Delegate',
            'source': 'farcaster_linked_address'
        })
    
len(ATTESTERS)

562

In [10]:
mapped_roles = {
    'Badgeholder Reviewer': None,
    
    'ACC (Second half)': 'Anticapture Commission',
    'Anticapture Commission Member': 'Anticapture Commission',
    'Anticapture Commission Lead': 'Anticapture Commission',
    
    'Code of Conduct Council': 'Code of Conduct',
    'Code of Conduct Council Lead': 'Code of Conduct',
    'Code of Conduct Council Member': 'Code of Conduct',
    
    'Developer Advisory Board Lead': 'Developer Advisory Board',
    'Developer Advisory Board Member': 'Developer Advisory Board',
    
    'Optimism Grants Council Lead': 'Grants Council',
    'Optimism Grants Council Member': 'Grants Council',
    'DeFi Committee Group A': 'Grants Council',
    'DeFi Committee Group B': 'Grants Council',
    'DeFi Committee Group C': 'Grants Council',
    'NFT and Gaming Committee': 'Grants Council',    
    'Tooling and Infrastructure Committee': 'Grants Council',    
    
    'Security Council Lead': 'Security Council',
    'Security Council Member': 'Security Council',
    
    'Token House Feedback Commission': 'Feedback Commission',
    'Citizens House Feedback Commission': 'Feedback Commission'
}
df_gov_councils = pd.DataFrame(gov_council_attestation_data)
df_gov_councils = df_gov_councils[
    (df_gov_councils['attester'] == '0xE4553b743E74dA3424Ac51f8C1E586fd43aE226F')
    & (df_gov_councils['govSeason'].isin(['5','6']))
]
df_gov_councils['recipient'] = df_gov_councils['recipient'].str.lower()
df_gov_councils['governance_council'] = df_gov_councils['govRole'].map(mapped_roles)

df_gov_councils[df_gov_councils['governance_council'].isna()==True]['govRole'].value_counts()

govRole
Badgeholder Reviewer    13
Name: count, dtype: int64

In [11]:
df_attesters = (
    pd.DataFrame(ATTESTERS)
    .groupby(['address', 'farcaster_id', 'farcaster_username'])['governance_role']
    .agg(set)
    .reset_index()
    .set_index('address')
    .join(
        df_gov_councils
        .groupby('recipient')['governance_council']
        .apply(set)
    )
    .sort_values(by='governance_council')
    .reset_index()
    .rename(columns={'index': 'address'})
)

df_attesters.to_csv("data/_local/governance_users.csv")
df_attesters

,address,farcaster_id,farcaster_username,governance_role,governance_council
0,0x07fda67513ec0897866098a11dc3858089d4a505,318387,v3naru,{Citizen},{Grants Council}
1,0xe93d59cc0bcecfd4ac204827ef67c5266079e2b5,,,{Top 100 Delegate},{Anticapture Commission}
2,0xe422d6c46a69e989ba6468ccd0435cb0c5c243e3,195722,joanbp,{Citizen},{None}
3,0xdc0a92c350a52b6583e235a57901b8731af8b249,247606,cpstl,{Citizen},{None}
4,0x8c580556fdb1f57853e49f409ae9b89f7658e7a2,7255,wish,{Top 100 Delegate},{Anticapture Commission}
...,...,...,...,...,...
222,0xf51c53b2c184aa43a7d24aa4f0cf13d0e8b56c51,10572,zemse,{Top 100 Delegate},NaN
223,0xf86a7a5b7c703b1fd8d93c500ac4cc75b67477f0,2,v,{Citizen},NaN
224,0xf9551c66995ed3ff9bb05c9fd7ff148bd75dc99a,5782,nathanvdh,{Top 100 Delegate},NaN
225,0xfb0a98121c3939657e59f835c66cfe20abb0cd4f,233135,krzkaczor,{Citizen},NaN


In [12]:
df_mgl = pd.read_csv('data/_local/governance_infra_and_tooling_with_badgeholder.csv', index_col=0)
df_mgl.set_index('attestationUID', inplace=True, drop=True)
mgl_uids = list(df_mgl.index)
df_mgl.tail(1)

,userfid,ethaddress,projectName,category,subcategory,ecosystem,likely_to_recommend,feeling_if_didnt_exist,explanation,private_feedback,createdAt,contribution,Badgeholder
attestationUID,,,,,,,,,,,,,
0x53478a97fec9fa86429c0bfded6b19a7e9b3b13b1736b2d2279d31f5f02ef4ac,9193,0x0000000000000000000000000000000000000000,Superfluid,Governance,Infra & Tooling,Optimism,7,Neutral,I have only had limited interaction with it (a...,NaN,2024-10-28 02:46:52.4753,Superfluid,True


In [13]:
df = pd.DataFrame(impact_attestation_data)
df['farcasterID'] = df['farcasterID'].apply(convert_farcaster)
df['farcasterUsername'] = df['farcasterID'].map(FARCASTER_USERS)
df['timeCreated'] = df['timeCreated'].apply(lambda x: datetime.datetime.utcfromtimestamp(x))

def get_name(metadata):
    if metadata.get('projectName'):
        return metadata.get('projectName')
    elif metadata.get('project'):
        return metadata.get('project').get('name')
    else:
        return None

df = df[df.id.isin(mgl_uids)]
df['mgl_project_name'] = df['metadata'].apply(get_name)

# provided by MGL team
overrides = {
    "0x8baac027df9c78c3dbda664c8f0190391e6b4bdabf240da9ccc4f0837933d8f2": "0x7753d360506e1d538e9a4720677595b2b811b3a3e74b81bd54c52ddfaaf51537", # MGL
    "0xfbca8cf7af5e74ffe1aea1f6850d0a427fb77286db170e478aca89f09b68e57c": "0xbb1e4aadda991e93d4ba4630582f222b2c57c0b0d729e29247cf2a3873936808", # Tribuni
    "0xf7248ad55667beaa6babe4700b52f5c868f60bdcb0e552f31369f78a22c7f1a6": "0xa88844cea135382e3484e39c3172033437121b35ca0bc8b10b9b8253984876b5", # EAS
    "0xdabe2dc1924e4422a8c9bc6ca7192666809579eb00c10e09360d5c4faa298001": "0x2aed462c71fc322e9b783e4b4454cdd01eb1909cdaf85f32ce70e9e5f3dc0f33", # Delegate Match
}
df['agoraProjectRegUID'] = df['projectRegUID'].apply(lambda x: overrides.get(x,x))

df['agora_project_name'] = df['agoraProjectRegUID'].map(AGORA_NAMES)

def get_scores(metadata):
    impact_eval_questions = metadata.get('impactAttestations', [])
    nps = pmf = None
    for q in impact_eval_questions:
        val = q.get('value')
        if not val:
            continue
        if q['name'] == 'Likely to Recommend':
            nps = pd.to_numeric(val)
        elif q['name'] == 'Feeling if didnt exist':
            pmf = pd.to_numeric(val)
    return {
        "NPS": nps,
        "PMF": pmf
    }

df['nps_score'] = df['metadata'].apply(lambda x: get_scores(x)['NPS'])
df['pmf_score'] = df['metadata'].apply(lambda x: get_scores(x)['PMF'])

roles = df_attesters.groupby('farcaster_id')['governance_role'].agg(lambda x: set.union(*x)).to_dict()
councils = (
    df_attesters[['farcaster_id', 'governance_council']]
    .dropna()
    .groupby('farcaster_id')['governance_council']
    .agg(lambda x: set.union(*x)).to_dict()
)
councils = {key: value for key, value in councils.items() if value != {None} and key != ''}
df['governance_role'] = df['farcasterID'].map(roles)
df['is_citizen'] = df['governance_role'].apply(lambda x: isinstance(x, set) and 'Citizen' in x)
df['is_top_delegate'] = df['governance_role'].apply(lambda x: isinstance(x, set) and 'Top 100 Delegate' in x)
df['governance_membership'] = df['farcasterID'].map(councils)

df.tail(1)

,id,attester,recipient,timeCreated,contributionRegUID,projectRegUID,farcasterID,issuer,metadataurl,metadata,farcasterUsername,mgl_project_name,agoraProjectRegUID,agora_project_name,nps_score,pmf_score,governance_role,is_citizen,is_top_delegate,governance_membership
863,0xfb3b6f3ced05253bfa4c528ff755d7a04d0a5474d120...,0x7484aABFef9f39464F332e632047983b67571C0a,0x07Fda67513EC0897866098a11dC3858089D4A505,2024-10-27 16:18:23,0xfda44b77d881a2c02dee8a5ab2e83f5a13ec91fc1fff...,0xf718cb944a663318db4e425bfb8aeed45b3718e6cf9f...,11596,MGL,https://gateway.pinata.cloud/ipfs/QmRECrJ9ZQfn...,"{'id': 2032, 'project': {'name': 'OP Passport'...",launamu,OP Passport,0xf718cb944a663318db4e425bfb8aeed45b3718e6cf9f...,OP Passport,4.0,1.0,"{Citizen, Top 100 Delegate}",True,True,NaN


In [14]:
df_badgies = df.set_index('id').join(df_mgl)[['farcasterID', 'userfid','farcasterUsername', 'Badgeholder', 'is_citizen', 'is_top_delegate']]
df_badgies['match'] = df_badgies['Badgeholder'] == df_badgies['is_citizen']
df_badgies[df_badgies['match']==False].drop_duplicates()

,farcasterID,userfid,farcasterUsername,Badgeholder,is_citizen,is_top_delegate,match
id,,,,,,,
0xd01e1e803d3c1b116d090d0a76b97e15bbdfc24a094392ab530d1cc98ebd0e6f,300324,300324,theethernaut,False,True,True,False
0x1074125ee28229dfbf4952f6c4d7f9f911fa7155921e868b06d1ded7aae1a1f0,777864,777864,None,True,False,False,False
0x0d6c857e50ede71745bf0ed5a2d2e64bea4fed648c1fba72e85ccaa290b2e842,12690,12690,owocki,True,False,False,False
0x13291b793c1c7ad376b7cb857ab7b66de58edcef9c719c12e8f0fbaa3298ab3e,18507,18507,publicworks,False,True,False,False
0x2bed302cf6a0635cf81d071b9d31c8f953ea4f454011a769d2672e09469e5f1f,811331,811331,elenoosh,False,True,False,False
0x2be6fb826740f15121859e7e2bd879f93a64ed91652ac16beae6141e23955bdb,472,472,None,True,False,False,False
0x6a30b1c66381a7640c01fca33c0cc5a7fab3317b43e26c11adee9fbbdf08c989,20540,20540,cryptochica.eth,True,False,False,False


# Create attestations dataframe

In [15]:
valid_attestations = (
    (df['agora_project_name'].isna()==False)
    & (df['governance_role'].isna()==False)
    & (df['nps_score'].isna() == False)
)
df_filtered = df[valid_attestations]
df_filtered['agora_project_name'].value_counts()

agora_project_name
Impact Evaluation Framework + Metrics Garden Database    27
Snapshot                                                 25
Pairwise                                                 15
Superfluid                                               14
Ethereum Attestation Service (EAS)                       14
                                                         ..
Ethereans Protocol                                        1
Superchain Attestation Explorer                           1
FairSharing                                               1
Vault                                                     1
ber4mins                                                  1
Name: count, Length: 61, dtype: int64

In [16]:
df_normalized = df.copy()

df_normalized['valid_attestation'] = valid_attestations

cols = [
    'timeCreated', 'projectRegUID', 'mgl_project_name', 'agoraProjectRegUID', 'agora_project_name',
    'farcasterUsername', 'farcasterID', 'is_citizen', 'is_top_delegate', 'governance_role',
    'governance_membership', 'nps_score', 'pmf_score', 'valid_attestation',
    'id', 'attester', 'recipient', 'contributionRegUID', 'metadataurl', 'metadata'
]

df_normalized = df_normalized[cols]
df_normalized.sort_values(by='timeCreated', ascending=False, inplace=True)
df_normalized.to_csv("data/normalized_attestations.csv")  

# Implement metrics on relevant attestations

In [17]:
metrics = calculate_metrics(df_filtered)

with open("data/attestation_metrics.json", "w") as f:
    json.dump(metrics,f,indent=2)

# Create dummy attestations

In [18]:
import random
import secrets

In [19]:
ATTESTERS = {}
for _, a in df_attesters.iterrows():
    addr = a['address']
    if addr in ATTESTERS:
        continue
    ATTESTERS.update({
        addr: {
            'is_citizen': 'Citizen' in a['governance_role'],
            'is_top_delegate': 'Top 100 Delegate' in a['governance_role'],
            'governance_membership': a['governance_council'] if isinstance(a, set) else {}
        }
    })

In [20]:
NUM_PROJECTS = 30
NUM_ATTESTERS = 100
NUM_ATTESTATIONS = 1000
PMF_SCALE = 3
NPS_SCALE = 10

projects = {
    p: random.random()
    for p in random.choices(PROJECTS, k=NUM_PROJECTS)
}

attesters = {
    addr:ATTESTERS[addr]
    for addr in random.choices(list(ATTESTERS.keys()), k=NUM_ATTESTERS)
}

dummy_attestations = []
for i in range(NUM_ATTESTATIONS):
    random_project = random.choice(list(projects.keys()))
    random_attester = random.choice(list(attesters.keys()))
    project_impact = projects.get(random_project)
    pmf_score = random.randint(int(project_impact*PMF_SCALE), PMF_SCALE)
    nps_score = random.randint(int(project_impact*NPS_SCALE), NPS_SCALE)
    dummy_attestations.append({
        'id': f"0x{secrets.token_hex(32)}",
        'attester': random_attester,
        'projectRegUID': random_project,
        'pmf_score': max(1,pmf_score),
        'nps_score': max(1,nps_score),
        **attesters[random_attester]
    })

In [21]:
df_dummy = pd.DataFrame(dummy_attestations)
dummy_metrics = calculate_metrics(df_dummy)

In [22]:
with open("data/dummy_metrics_v2.json", "w") as f:
    json.dump(dummy_metrics, f, indent=2)